# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

# Set path
project_root = os.path.abspath(os.path.join("..", "src"))
if project_root not in sys.path:
    sys.path.append(project_root)

from data_processing import data_updater_v01 as du

In [2]:
date = '20131018'
routes_txt, trips_txt, shapes_txt, calendar_txt, calendar_dates_txt = du.load_txt_data(date)

In [3]:
data_folder = '../data/processed/'
shapes_df, routes_df, route_versions_df, shape_variants_df, shape_variant_activations_df, temporary_changes_df = du.load_df_data(data_folder)

In [4]:
trips_df = trips_txt.copy()
calendar_df = calendar_txt.copy()
calendar_dates_df = calendar_dates_txt.copy()

In [5]:
unique_services = trips_df["service_id"].unique()
trip_dates = {
    service: du.get_active_dates(calendar_df, service)
    for service in unique_services
}
trip_first_date = {
    service: dates[0] if dates else None
    for service, dates in trip_dates.items()
}

In [6]:
latest_routes_df = du.trips2latest_routes(trips_df, trip_first_date, routes_txt)

In [7]:
updated_routes_df = du.update_routes_df(routes_df, latest_routes_df)
du.save_routes(updated_routes_df, data_folder)

No duplicate route_id found in routes_df.
routes_df saved to ../data/processed/routes.csv


In [8]:
extended_route_versions_df = du.update_route_versions(route_versions_df, latest_routes_df, date)
du.save_route_versions(extended_route_versions_df, data_folder)
#extended_route_versions_df

routes_df saved to ../data/processed/route_versions.csv


In [9]:
df_noexceptions = du.get_df_with_noexception(trip_dates, trips_df)
#df_noexceptions

In [10]:
df_exceptions = du.get_df_with_exception(calendar_dates_df, trips_df)
#df_exceptions

In [11]:
return_df = du.get_extended4shape_variants(extended_route_versions_df, df_noexceptions, df_exceptions)

7629 duplikált sort eltávolítottunk, ahol csak az exception_type tért el (NaN vs nem-NaN).


In [12]:
shape_variants_df, shape_variant_activations_df = du.update_shape_variants_and_activations(return_df, shape_variants_df, shape_variant_activations_df)

Updated shape_variants_df:
Shape: (1151, 6)

Updated shape_variant_activations_df:
Shape: (38754, 3)

Summary:
Total unique shape variants: 1151
Total shape variant activations: 38754
No new variants added
No new activations added


In [13]:
du.save_shape_variants_df(shape_variants_df, data_folder)
du.save_shape_variant_activations_df(shape_variant_activations_df, data_folder)

shape_variants_df saved to ../data/processed/shape_variants.csv
shape_variant_activations_df saved to ../data/processed/shape_variant_activations.csv
